# Temperature Time Series

# Part One:  Exploration of Canadian Temperature 
## Getting the Canadian Temperature Data

This is the first part in the series of analysing temperature in Canadian series. This notebook focus only on getting the time series. It can be considered as a basic intro but several aspects are covered. This notebook only focus on one city as well. The principle derived here applies to most of the other cities

# Temperature Time Series Visualization

$\mathbf{Time\ series}$ generally refers to data collected over time intervals. Its dependence on time breaks the independence variable assumption used in regression analysis.
This tutorial aims to show some necessary and essential steps taken before performing  a time series of $\textit{visualization and forcasting}$. Exploration deals with idensify features and visualization of the time series, while forecasting refers to making predictions for future trends. It could be in the market, stock price, accidents, the number of patients arriving at a clinic. Most of the concepts presented here equally apply to most time series data. 

In this set of not books on exploration notebooks, however, we will be working exclusively with temperature data. The data is from the NOAA weather station, and we will pick one station in Prince Albert, a city in SK, Canada. The data, as well as its properties, can be found in the [NOAA website](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ "NOAA website"). There are three other stations in Canada, but we will focus on this one which is part of the GCOS Surface Network (GSN). In the next project, we will work with multiple cities.

## 1. Overview of notebooks to follow
First, note that this notebook follows a similar approach to that in ref [here](https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/?utm_source=blog&utm_medium=stockmarketpredictionarticle).It is also an introductory article on straightforward concepts about time series which will cover aspects like

   - Learn the steps to create a Time Series dataset from the NOAA data
   - Display some features of raw data
   - Prepare or clean the information

In [1]:
import numpy as np
from datetime import datetime
#used to get data directly from the URL
import urllib.request

Changing the size of the default parameter sizes

## 2. Getting the data
The data as menntion is downloaded from [NOAA link here](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt).
 - Loading and Handling Time Series in Pandas
 - How to Check Stationarity of a Time Series?
 - How to make a Time Series Stationary

In [2]:
#the data of the stations are stored in stations
urllib.request.urlretrieve('ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt','stations.txt')

('stations.txt', <email.message.Message at 0x7f6450034940>)

check the available list of stations that stores the data

In [3]:
open('stations.txt','r').readlines()[:8]

['ACW00011604  17.1167  -61.7833   10.1    ST JOHNS COOLIDGE FLD                       \n',
 'ACW00011647  17.1333  -61.7833   19.2    ST JOHNS                                    \n',
 'AE000041196  25.3330   55.5170   34.0    SHARJAH INTER. AIRP            GSN     41196\n',
 'AEM00041194  25.2550   55.3640   10.4    DUBAI INTL                             41194\n',
 'AEM00041217  24.4330   54.6510   26.8    ABU DHABI INTL                         41217\n',
 'AEM00041218  24.2620   55.6090  264.9    AL AIN INTL                            41218\n',
 'AF000040930  35.3170   69.0170 3366.0    NORTH-SALANG                   GSN     40930\n',
 'AFM00040938  34.2100   62.2280  977.2    HERAT                                  40938\n']

We will look for Canadian stations with GSN. The code name of stations in Canada starts with 'CA'

In [4]:
Canada_stations = {}
for line in open('stations.txt','r'):
    if (line[:2] == 'CA') and ('GSN' in line):
        fields = line.split()
        
        Canada_stations[fields[0]] = ' '.join(fields[4:])

We can check a few stations

In [5]:
def findstation(s):
    found = {code: name for code,name in Canada_stations.items() if s in name}
    print(found)

In [6]:
findstation('PRINCE')

{'CA004056240': 'SK PRINCE ALBERT A GSN 71869'}


We will be working with Prince Albert data set. There is nothing special about Prince Albert. One can pick any city. The station ID is CA004056240. It is used below to get the data.

In [7]:
urllib.request.urlretrieve('https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/gsn/CA004056240.dly', 'CA004056240.dly')

('CA004056240.dly', <http.client.HTTPMessage at 0x7f6450065588>)

In [8]:
open('CA004056240.dly','r').readlines()[:5]

['CA004056240194212TMAX -150  C -100  C  -83  C  -94  C -139  C -161  C -194  C -211  C -178  C -217  C -206  C  -72  C   -6  C  -94  C  -61  C  -94  C -189  C -283  C -194  C -161  C -139  C -150  C -156  C -128  C -156  C -183  C  -61  C  -94  C -261  C -206  C -256  C\n',
 'CA004056240194212TMIN -233  C -222  C -200  C -167  C -194  C -206  C -339  C -306  C -278  C -306  C -328  C -311  C -167  C -117  C -189  C -161  C -217  C -350  C -356  C -239  C -183  C -256  C -233  C -317  C -311  C -306  C -300  C -244  C -328  C -317  C -333  C\n',
 'CA004056240194212PRCP    0T C   18  C   23  C   10  C    0T C    0T C    0T C    0  C   13  C    0  C    0  C    8  C    3  C    8  C    8  C   58  C    3  C    0  C    5  C    5  C    3  C   69  C   10  C    0  C    0  C    0  C   18  C    0T C    3  C    8  C    5  C\n',
 'CA004056240194212SNOW    0T C   18  C   23  C   10  C    0T C    0T C    0T C    0  C   13  C    0  C    0  C    8  C    3  C    8  C    8  C   58  C    3  C    0  C    5

We can see that the data doesn't make so much sense at a first glance. To make sense of the data, there is a [readme.txt](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) on the website. Basically, ${-999}$ are NANs. We Below we read a description of a segment of the readme.txt

In [9]:
urllib.request.urlretrieve('https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt', 'readme.txt')
open('readme.txt','r').readlines()[108:128]

['------------------------------\n',
 'Variable   Columns   Type\n',
 '------------------------------\n',
 'ID            1-11   Character\n',
 'YEAR         12-15   Integer\n',
 'MONTH        16-17   Integer\n',
 'ELEMENT      18-21   Character\n',
 'VALUE1       22-26   Integer\n',
 'MFLAG1       27-27   Character\n',
 'QFLAG1       28-28   Character\n',
 'SFLAG1       29-29   Character\n',
 'VALUE2       30-34   Integer\n',
 'MFLAG2       35-35   Character\n',
 'QFLAG2       36-36   Character\n',
 'SFLAG2       37-37   Character\n',
 '  .           .          .\n',
 '  .           .          .\n',
 '  .           .          .\n',
 'VALUE31    262-266   Integer\n',
 'MFLAG31    267-267   Character\n']

## 3. Cleaning/Formatting the data
We definitely need a parser the data. We can note from the data that, each record for example $\textit{open('CA004056240.dly','r').readlines()[0] }$ is a string. The first entry is a combination of the station name and year, month, and the type of measurement (TMIN, TMAX etc). Below is a parser to convert the data into a record array of multiple types.

The function $\text{parsefile}$ helps us read each record from the the station. The parameters specify how the record is read. Let's go through each parameter at a time.

$\textbf{dly_delimiter}$: A quick check [online](https://docs.scipy.org/doc/numpy/reference/generated/numpy.genfromtxt.html) provides the info that is is where the read breaks. Lets take a look at the first record which is _'CA004056240194212TMAX -150  C -100  C  -83  C  -94  C -139  C -161  C -194  C -211  C -178  C -217  C -206  C  -72  C   -6  C  -94  C  -61  C  -94  C -189  C -283  C -194  C -161  C -139  C -150  C -156  C -128  C -156  C -183  C  -61  C  -94  C -261  C -206  C -256  C\n'_ First 11 chars, values specifies the station code, next 4 the year, next 2 the month, next 4 the measurement type, then the other values are temperatures and it units C for the next 31 days. After the measurement type, 5 chars (including the space) contains the temperature, and the other 3 (space, C, space) are also read. This is repeated 31 times.

$\textbf{dly_usecols}$: Specify the columns which are used in defining the data. The station code and each of the (_c_) are not used.

$\textbf{dly_dtype}$: The datatypes of each of the columns. We see that the first two the year and month are int while the measurement type is str

$\textbf{dly_names}$: These are names that are used to access each of the columns. The values are specified below

In [10]:
def parsefile(filename):
    '''
    Takes a file and parse it to return a numpy array
    '''
    return np.genfromtxt(filename,
                         delimiter = dly_delimiter,
                         usecols = dly_usecols,
                         dtype = dly_dtype,
                         names = dly_names)

In [11]:
dly_delimiter = [11,4,2,4] + [5,1,1,1] * 31 
dly_usecols = [1,2,3] + [4*i for i in range(1,32)]
dly_dtype = [np.int32,np.int32,(np.str_,4)] + [np.int32] * 31
dly_names = ['year','month','obs'] + [str(day) for day in range(1,31+1)]

Let's parse Prince Albert's temperature data and check these

In [12]:
Prince_Albert = parsefile('CA004056240.dly')
print(Prince_Albert[0])
print('------------------------------Year, month---------------------------------------------------------')
print('Year: ', Prince_Albert[0]['year'], 'Month: ',Prince_Albert[0]['month'])

(1942, 12, 'TMAX', -150, -100, -83, -94, -139, -161, -194, -211, -178, -217, -206, -72, -6, -94, -61, -94, -189, -283, -194, -161, -139, -150, -156, -128, -156, -183, -61, -94, -261, -206, -256)
------------------------------Year, month---------------------------------------------------------
Year:  1942 Month:  12


In [13]:
measurements = set(Prince_Albert['obs']) #Check unique values
measurements

{'PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN'}

Multiple observation are present, namely 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX' and 'TMIN'. For now, lets focus on the maximum and minimum temperatures. The data is not in the desired format, so lets rearrange it including the dates.

In [14]:
def unroll(record):
    '''
    Takes a station temperature data, arrange it as an array of pairs of daily temperature and value
    record: input temperature data
    Pameters:
    -----------
    startdate: the date (yr, m) to start considering
    dates: Hold a date in daily steps
    '''
    startdate = np.datetime64('{}-{:02}'.format(record['year'],record['month']))
    dates = np.arange(startdate, startdate + np.timedelta64(1,'M'), np.timedelta64(1,'D')) #Daily step increment
    
    rows = [(date, record[str(i+1)]/10) for i, date in enumerate(dates)]
    
    return np.array(rows, dtype=[('date','M8[D]'), ('value','d')])

The data is then grouped by day and value

In [15]:
def getobs(filename, obs):
    '''
    Parse a record and each line contain montly record that are unrolled and concated to the data variable
    Parameters:
    ---------------
    filename: specific  record
    obs: is the observation of choice eh TMAX, TMIN etc

    '''
    data = np.concatenate([unroll(row) for row in parsefile(filename) if row[2] == obs])
    
    data['value'][data['value'] == -999.9] = np.nan
    
    return data

In [16]:
Prince_Albert_tmax = getobs('CA004056240.dly','TMAX')
Prince_Albert_tmin = getobs('CA004056240.dly','TMIN')

There are some null values. We will replace them by avarage of the two neigbouring values

In [17]:
np.mean(Prince_Albert_tmax['value']), np.mean(Prince_Albert_tmin['value'])

(nan, nan)

In [18]:
Prince_Albert_tmax[np.isnan(Prince_Albert_tmax['value'])].size
print('Number of entries: ', len(Prince_Albert_tmax)) # Sizetime series
print('Missing values: {} '.format(Prince_Albert_tmax[np.isnan(Prince_Albert_tmax['value'])].size)) # Missing values

Number of entries:  28002
Missing values: 1199 


This data shows that the missing values are around the same period but not the same. This notebook is meant to demonstrate basic concepts in time series; we will only use the one of the time series. There are many efficient ways to fill the nans, but for not we will fill them with an interpolation. Since the temperatures are seasonal, before training a model, we can instead fill with average temperatures from other years. 

In [19]:
def fillnans(data):
    '''
    Basically, the nans are filled by the average of the neighbouring two values
    '''
    dates_float = data['date'].astype(np.float64)    
    nan = np.isnan(data['value'])    
    data['value'][nan] = np.interp(dates_float[nan], dates_float[~nan], data['value'][~nan])

In [20]:
fillnans(Prince_Albert_tmax)
fillnans(Prince_Albert_tmin)

In [26]:
(Prince_Albert_tmax.shape, Prince_Albert_tmax.shape)

((28002,), (28002,))

In [21]:
np.mean(Prince_Albert_tmax['value']), np.mean(Prince_Albert_tmin['value'])

(7.1800103564031135, -5.482544103992572)